# Deconvolution study

In [183]:
Pkg.add("PrettyPrinting")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
   Updating registry at `~/.julia/registries/JuliaComputingRegistry`
  Resolving package versions...
  Installed PrettyPrinting ─ v0.2.1
Updating `~/JuliaProjects/JImages/Project.toml`
  [54e16d92] + PrettyPrinting v0.2.1
Updating `~/JuliaProjects/JImages/Manifest.toml`
  [54e16d92] + PrettyPrinting v0.2.1


In [184]:
using OffsetArrays
using StaticArrays
using Images
using FileIO
using Printf
using PrettyPrinting

┌ Info: Precompiling PrettyPrinting [54e16d92-306c-5ea0-a30b-337be88ac337]
└ @ Base loading.jl:1278


## Basis

### Spatial Filter

A spatial filter consists on a *neighborhood* (tipically a small window, say 3x3 of 5x5 around the pixel) and a *predefined operation* which is performed on the image pixels encompassed by the neighborhood. Filtering creates a new pixel with coordinates equal to the coordinates of the center of the neighborhood and whose value is the filtering operation. A *filtered* image is generated as the center of the filter visits each pixel in the input image. 

<img src="SpatialFiltering.png">

In [183]:
#display(FileIO.load("SpatialFiltering.png"))

### Spatial correlation and convolution

Correlation and convolution are closely related concept. *Correlation is the process of moving a filter mask over the image and computing the sum of products at each location*. The mechanics of convolution are the same, except that the filter is first rotated by 180°. For symmetric filters both operations are the same.

<img src="corrAndConv.png"> 

The figure shows both operations in the case of a 1D function *f* and filter *w*. 
- (a) shows the f and w
- (b) shwows the starting position for corr (left) or conv (right). Notice that there are parts of the function that do not overlap. The solution to this problem is to pad f with enough 0s on either side to allow each pixel in w to visit every pixel in f. If the filter is of size m, we need m - 1 0s on either side of f. 
- (c) shows a padded function.
- The process of sliding the filter over f is shown in d through g. The first value of correlation is the sum of products of f and w for the initial position shown in c (the sum of products is 0). This corresponds to a displacement x = 0. To obtain the second value of correlation, we shift w one pixel location to the right (a displacement of x = 1) and compute the sum of products. The result again is 0. In fact, the first nonzero result is when x = 3, in which case the 8 in w overlaps the 1 in f and the result of correlation is 8. 
- Proceeding in this manner, we obtain the full correlation result in Fig. (g). Note that it took 12 values of x (i.e., x = 0, 1, 2, ..., 11) to fully slide w past f so that each pixel in w visited every pixel in f.

- correlation is a function of displacement of the filter. IThe first value of correlation corresponds to zero displacement of the filter, the second corresponds to one unit displacement, and so on. 
- correlating a filter w with a function that contains all 0s and a single 1 yields a result that is a copy of w , but rotated by 180°. We call a function that contains a single 1 with the rest being 0s a discrete unit impulse. So we conclude that correlation of a function with a discrete unit impulse yields a rotated version of the function at the location of the impulse.

- Instead a fundamental property of convolution is that convolving a function with a unit impulse yields a copy of the function at the location of the impulse. 
- To perform convolution all we do is rotate one function by 180° and perform the same operations as in correlation. As it turns out, it makes no difference which of the two functions we rotate.

<img src="corrAndConv2D.png">  

Extension to images (2D) is shown in the figure: f(x,y) is a 5x5 function and w(x,y) is a 3x3 filter. 
- We start by padding each dimension with 3-1 = 2 zeros in each side. Thus the padded image is 9 x 9. For a filter of size m * n, we pad the image with a minimum of m-1 rows of 0s at the top and bottom and n-1 columns of 0s on the left and right. In (a) the filter is in the initial position.
- (b) shows the correlation result. The first element different from zero is 9. Then, as we slide the window we obtain a rotated copy of w. Finally we crop to the dimensions of f. 
- (c) shows how to make the convolution. We need to *flip* the filter, changing the order of files and columns (the equivalent of rotatin the filter by 180 degrees). 
- (d) shows that the result of the convoltuion is a copy of the initial filter, with the dimension, in (e) of the image f (after cropping). 

The *correlation* of a filter w(x, y) of size m * n with an image f(x, y), denoted as $w(x, y) \star f(x, y)$, is:
$$
w(x,y) \star f(x,y) = \sum_{s = -a}^{a} \sum_{t = -b}^{b} w(s,t) f(x + s, y + t) 
$$

While  the *convolution* of a filter w(x, y) of size m * n with an image f(x, y), denoted as $w(x, y) \ast f(x, y)$, is:
$$
w(x,y) \ast f(x,y) = \sum_{s = -a}^{a} \sum_{t = -b}^{b} w(s,t) f(x-s, y-t) 
$$

Flipping and shifting f (in convolution) instead of w is done for notational simplicity. The equations are evaluated for all values of the displacement variables x and y so that every element of w visits every pixel in f, which we assume has been padded appropriately.

### Example: Generating a gaussian filter (or kernel)

A Gaussian function of two variables, with mean $\mu=0$ and std $\sigma$ has the  form

$$
g(x,y) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{x^2 + y^2}{2\sigma^2}}
$$

To generate, say, a 3x3 filter we sample it about its center. Thus, $w_1 = h(-1, -1), w_2 = h(-1, 0), ..., w_9 = h(1, 1)$. 


## Application

### Functions

`pretty_print` formats arrays in compact "pretty" form

In [251]:
function pretty_print(fc::AbstractArray)
    r,c = size(fc)
    for i in 1:r 
        e1 = ""
        for j = 1:c
            e1 = string(e1, "\t", @sprintf("%d", fc[i,j]))
        end
        println(e1)
    end
end

function pretty_print(ff::OffsetArray)
    fc = collect(ff)
    pretty_print(fc, form)
end

function pretty_print(CI::CartesianIndices)
    r,c = size(CI)
    for i in 1:r 
        e1 = ""
        for j = 1:c
            x,y = CI[i,j].I
            e1 = string(e1, "\t", @sprintf("(%d,%d)", x,y))
        end
        println(e1)
    end
end


function ci_array(ranges = (-1:1, -1:1))
    broadcast(i->i.I, CartesianIndices(ranges))
end

function flip2d(x::Array{Float64,2})
    y = copy(x)
    for d in 1:2
        y = reverse(y, dims=d)
    end
    return y
end

flip2d (generic function with 1 method)

### Cartesian indices

A `CartesianIndex{N}` represents an N-dimensional index. `CartesianIndexes` are based on tuples, and indeed you can access the underlying tuple with `Tuple(i)`.
A `CartesianIndices`  acts like an array of CartesianIndex values. Thus a Cartesian index (N=2) maps a point in a grid, and is very well suited to describe 2D images. Something applies to higher dimensions. 

In [244]:
A = [[5., 6., 7.] [1., 2., 3.] [10.,20., 30.,] [100., 200., 300.]]

3×4 Array{Float64,2}:
 5.0  1.0  10.0  100.0
 6.0  2.0  20.0  200.0
 7.0  3.0  30.0  300.0

In [245]:
size(A)

(3, 4)

In [246]:
pretty_print(A)

	5	1	10	100
	6	2	20	200
	7	3	30	300


In [175]:
size(CartesianIndices(A))

(3, 4)

In [196]:
i = CartesianIndices(A)

3×4 CartesianIndices{2,Tuple{Base.OneTo{Int64},Base.OneTo{Int64}}}:
 CartesianIndex(1, 1)  CartesianIndex(1, 2)  …  CartesianIndex(1, 4)
 CartesianIndex(2, 1)  CartesianIndex(2, 2)     CartesianIndex(2, 4)
 CartesianIndex(3, 1)  CartesianIndex(3, 2)     CartesianIndex(3, 4)

In [197]:
typeof(i)

CartesianIndices{2,Tuple{Base.OneTo{Int64},Base.OneTo{Int64}}}

In [198]:
supertype(typeof(i))

AbstractArray{CartesianIndex{2},2}

In [199]:
pretty_print(i)

	(1,1)	(1,2)	(1,3)	(1,4)
	(2,1)	(2,2)	(2,3)	(2,4)
	(3,1)	(3,2)	(3,3)	(3,4)


In the example above, the cartesian indices are extracted from the array A, and thus are simply the conventional indexing. However, One can construct CartesianIndices around suitable axis. Thus, to generate a gaussian filter is convenient to use the notation introduced above, $w_1 = h(-1, -1), w_2 = h(-1, 0), ..., w_9 = h(1, 1)$. This can be trivially done with CartesianIndices

In [200]:
i = CartesianIndices(((-1:1), (-1:1)))

3×3 CartesianIndices{2,Tuple{UnitRange{Int64},UnitRange{Int64}}}:
 CartesianIndex(-1, -1)  CartesianIndex(-1, 0)  CartesianIndex(-1, 1)
 CartesianIndex(0, -1)   CartesianIndex(0, 0)   CartesianIndex(0, 1)
 CartesianIndex(1, -1)   CartesianIndex(1, 0)   CartesianIndex(1, 1)

In [201]:
iter[1]

CartesianIndex(-1, -1)

In [202]:
pretty_print(i)

	(-1,-1)	(-1,0)	(-1,1)
	(0,-1)	(0,0)	(0,1)
	(1,-1)	(1,0)	(1,1)


### Gaussian Kernel

- First, we define a generic 2D gaussian function. 

In [16]:
G(x,y,σ=1.0) = exp(-(x^2 + y^2) / (2 * σ^2) ) # not normalised

G (generic function with 2 methods)

- And now a specialization for Cartesian Indices. 

In [17]:
G(I::CartesianIndex{2}, σ=1.0) = G(I.I..., σ)  # I.I... matches the tuple I.I to values (x,y)

G (generic function with 4 methods)

#### Finally the function defining the Gaussian Kernel (Filter), which applies to each of the cartesian indices the value of G

In [18]:
function gaussian_filter(σ, l=1) # order "l=1" creates a 3 x 3 filter, l=2 a 5x5 filter and so on 
    gauss = map(i->G(i, σ), CartesianIndices((-l:l, -l:l))) 
    gauss./sum(gauss)  #divide all elements of gauss by its sum to normalise
end

gaussian_filter (generic function with 2 methods)

In [203]:
gaussian_filter(1.0, 1)

3×3 Array{Float64,2}:
 0.0751136  0.123841  0.0751136
 0.123841   0.20418   0.123841
 0.0751136  0.123841  0.0751136

In [204]:
sum(gaussian_filter(1.0, 1))

0.9999999999999999

In [205]:
gaussian_filter(0.5, 1)

3×3 Array{Float64,2}:
 0.0113437  0.0838195  0.0113437
 0.0838195  0.619347   0.0838195
 0.0113437  0.0838195  0.0113437

In [206]:
sum(gaussian_filter(0.5, 1))

1.0

In [207]:
gaussian_filter(1.0, 2)

5×5 Array{Float64,2}:
 0.00296902  0.0133062  0.0219382  0.0133062  0.00296902
 0.0133062   0.0596343  0.0983203  0.0596343  0.0133062
 0.0219382   0.0983203  0.162103   0.0983203  0.0219382
 0.0133062   0.0596343  0.0983203  0.0596343  0.0133062
 0.00296902  0.0133062  0.0219382  0.0133062  0.00296902

In [208]:
sum(gaussian_filter(1.0, 2))

0.9999999999999999

### Filter frame (neighboors)

To perform a convolution, we slide a frame of the size of the filter (we will assume it to be squared for simplicity, like `w` in the exapmple above) centered in each element of the matrix representing the function to be convoluted (`fxy`), then multiply the elements of the filter by the elements contained in the frame and sum the products, replacing the value in the center of the frame by the result of the operation.

Each element of `fxy` is labeled by a CartexianIndex (i,j). Thus, we can represent the frame surrounding this element by (i-w:i+w, j-w:j+w), where w is the integer divison of the filter size and 2. For a 3 x 3 filter, w = 1. Notice that the idea of the sliding frame is well defined for filters of odd dimensions. For filters of even dimensions one needs to pad the filter with extra zeros. 

In [211]:
@inline function filter_frame(I::CartesianIndex{2}, filter_size::Int) 
    # assume that the filter is squared
    w = filter_size÷2   
    ntuple(i->(I.I[i]-w): (I.I[i]+w), 2)  # for N=2 i runs (1,2)
end

filter_frame (generic function with 1 method)

In [25]:
@inline function neighbors(I::CartesianIndex{N}, l::Int) where N
    # if l = size(filter), w = l\div 2. e.g, size of filter = 3, w = 3 \div 2 = 1, range -1:1
    w = l÷2   
    ntuple(i->(I.I[i]-w): (I.I[i]+w), Val(N))  # for N=2 i runs (1,2)
end

neighbors (generic function with 1 method)

Define the filter `w` as in the example above:

In [217]:
w = collect(transpose(Float64.(reshape(1:9, 3, 3))))

3×3 Array{Float64,2}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

In [218]:
ci = CartesianIndices(w)

3×3 CartesianIndices{2,Tuple{Base.OneTo{Int64},Base.OneTo{Int64}}}:
 CartesianIndex(1, 1)  CartesianIndex(1, 2)  CartesianIndex(1, 3)
 CartesianIndex(2, 1)  CartesianIndex(2, 2)  CartesianIndex(2, 3)
 CartesianIndex(3, 1)  CartesianIndex(3, 2)  CartesianIndex(3, 3)

In [220]:
for i in ci
    println("For cartexian index = ", i)
    println("neighbors (w)  = ", neighbors(i, size(w,1)))
end

For cartexian index = CartesianIndex(1, 1)
neighbors (w)  = (0:2, 0:2)
For cartexian index = CartesianIndex(2, 1)
neighbors (w)  = (1:3, 0:2)
For cartexian index = CartesianIndex(3, 1)
neighbors (w)  = (2:4, 0:2)
For cartexian index = CartesianIndex(1, 2)
neighbors (w)  = (0:2, 1:3)
For cartexian index = CartesianIndex(2, 2)
neighbors (w)  = (1:3, 1:3)
For cartexian index = CartesianIndex(3, 2)
neighbors (w)  = (2:4, 1:3)
For cartexian index = CartesianIndex(1, 3)
neighbors (w)  = (0:2, 2:4)
For cartexian index = CartesianIndex(2, 3)
neighbors (w)  = (1:3, 2:4)
For cartexian index = CartesianIndex(3, 3)
neighbors (w)  = (2:4, 2:4)


### OffsetArrays

-OffsetArrays provides Julia users with arrays that have arbitrary indices

OA = OffsetArray(A, axis1, axis2, ...)

- This means that the indices in each dimension do not need to run like normal indices (1,2,3...) but can take, for instance a tuple of CartesianIndex. Thus, a 3x3 array would have elements running from 1:3 in each axis, while an OffsetArray could have elements running from -1:1 in each axis

In [221]:
w

3×3 Array{Float64,2}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

In [222]:
OA = OffsetArray(w, (-1:1, -1:1)) # OA will have axes (-1:1, -1:1)

3×3 OffsetArray(::Array{Float64,2}, -1:1, -1:1) with eltype Float64 with indices -1:1×-1:1:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

In [223]:
OA = OffsetArray(w, (0:2, 0:2)) # OA will have axes (0:2, 0:2)

3×3 OffsetArray(::Array{Float64,2}, 0:2, 0:2) with eltype Float64 with indices 0:2×0:2:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

### Padding an image

Suppose we want to pad an image with a border. Remember that for a filter of size m * n, we need to pad the image with a minimum of m-1 rows of 0s at the top and bottom and n-1 columns of 0s on the left and right. In (a) the filter is in the initial position. Let's assume for convenience that the filter is squared (mxm). Thus, if the image has size (s,l) an we define border = m-1 = 2, we need to pad with s + border (left/right), and s + border (top/bottom), so (s + 2 * border) in each axis. 

Suppose, as in our examples above that the image has size 5x5 and the filter has size 3x3. Then, m=2 and the passed image will have a size 5 + 2 * 2 = 9 in each axis. 

In [224]:
function pad(data, border)
    padded_size  = map(s->s+2*border, size(data))  # in each axis 
    
    # the padded_range is relative to the main image. 
    # We want to extend it to the left/up (1-border) and to the right/down (s+border)
    padded_range = map(s->(1-border):(s+border), size(data))
    
    # the range to index the output image extends from 1:s in each direction
    range        = CartesianIndices(map(s->1:s, size(data))) 
    
    # zeros dimension sxs and offset array 
    odata        = OffsetArray(zeros(eltype(data), padded_size), padded_range...)
    
    odata[range] = data # fill the padded image with the original data
     
    return odata, collect(range)
end

pad (generic function with 1 method)

Let's disect the function, using our example above. First we define f(x,y)

In [225]:
fxy = zeros(eltype(0.), (5,5))
fxy[3,3] = 1;

In [226]:
fxy  # this is the function f(x,y)

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

define also the filter

In [227]:
w 

3×3 Array{Float64,2}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

The border (number of padded raws/columns) for a m x m filter is m-1 in each axis

In [228]:
border = size(w,1)-1

2

The size of the padded image is

In [229]:
padded_size  = map(s->s+2*border, size(fxy))

(9, 9)

The padded range extends the size of the image to the left/right (up/down) by border. This can be expressed in the range

In [230]:
padded_range = map(s->(1-border):(s+border), size(fxy))

(-1:7, -1:7)

while the range of the image is:

In [231]:
range  = CartesianIndices(map(s->1:s, size(fxy)))

5×5 CartesianIndices{2,Tuple{UnitRange{Int64},UnitRange{Int64}}}:
 CartesianIndex(1, 1)  CartesianIndex(1, 2)  …  CartesianIndex(1, 5)
 CartesianIndex(2, 1)  CartesianIndex(2, 2)     CartesianIndex(2, 5)
 CartesianIndex(3, 1)  CartesianIndex(3, 2)     CartesianIndex(3, 5)
 CartesianIndex(4, 1)  CartesianIndex(4, 2)     CartesianIndex(4, 5)
 CartesianIndex(5, 1)  CartesianIndex(5, 2)     CartesianIndex(5, 5)

Notice that the image goes from 1:5 and the padded image will extend 2 in each direction. We can form now the padded function as an offset array. First, filled with zeros

In [232]:
padded_function   = OffsetArray(zeros(eltype(fxy), padded_size), padded_range...)

9×9 OffsetArray(::Array{Float64,2}, -1:7, -1:7) with eltype Float64 with indices -1:7×-1:7:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

And then, copy the fxy function in the center of the padded function

In [233]:
padded_function[range] = fxy

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [234]:
padded_function 

9×9 OffsetArray(::Array{Float64,2}, -1:7, -1:7) with eltype Float64 with indices -1:7×-1:7:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

This is what we get calling the pad function

In [235]:
fxyp, cr = pad(fxy, border);

In [236]:
fxyp

9×9 OffsetArray(::Array{Float64,2}, -1:7, -1:7) with eltype Float64 with indices -1:7×-1:7:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [237]:
cr

5×5 Array{CartesianIndex{2},2}:
 CartesianIndex(1, 1)  CartesianIndex(1, 2)  …  CartesianIndex(1, 5)
 CartesianIndex(2, 1)  CartesianIndex(2, 2)     CartesianIndex(2, 5)
 CartesianIndex(3, 1)  CartesianIndex(3, 2)     CartesianIndex(3, 5)
 CartesianIndex(4, 1)  CartesianIndex(4, 2)     CartesianIndex(4, 5)
 CartesianIndex(5, 1)  CartesianIndex(5, 2)     CartesianIndex(5, 5)

## Convolution 

### Convolve first version

- Start with an array of the same size of the padded array
- Loop over the indexes
- Compute the neighborhood of each index
- Take a view of the array in the neighborhood
- Multiply element by element the view and the filter and sum

In [239]:
function convolve1(I::CartesianIndex{2}, 
                   fxy::Array{Float64,2}, 
                   padf::OffsetArray, 
                   filter::Array{Float64,2})
    
    # start with an array of the size of fxy filled with zeros
    fc = similar(fxy)
    
    for i in range  # loop over the Cartesian Indices
        frame = filter_frame(i, size(filter, 1)) # take a frame
        d = view(padf, frame...) # and use the frame to obtain a view of padf
        fc[i] = sum(d .* filter) # replace element by sum of products
    end
    return fc
end

convolve1 (generic function with 3 methods)

In [249]:
fc1 = convolve1(cr, fxy, fxyp, w)

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  9.0  8.0  7.0  0.0
 0.0  6.0  5.0  4.0  0.0
 0.0  3.0  2.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0

#### NB! 
Our convolve1 function has performed a correlation, not a convolution! We can obtain the convolution flipping the result

In [248]:
ffc1 = flip2d(fc1)

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  1.0  2.0  3.0  0.0
 0.0  4.0  5.0  6.0  0.0
 0.0  7.0  8.0  9.0  0.0
 0.0  0.0  0.0  0.0  0.0

We obtain the same result flipping the kernel

In [250]:
fc2 = convolve1(cr, fxy, fxyp, flip2d(w))

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  1.0  2.0  3.0  0.0
 0.0  4.0  5.0  6.0  0.0
 0.0  7.0  8.0  9.0  0.0
 0.0  0.0  0.0  0.0  0.0

Notice that we have performed a correlation, not a convolution. To perform a convolution we need to flip the function of the kernel

Flipping the function 

### Convolve, second version

A more compact version of convolve takes advantage of broadcasting. The function is applied like this

`f_convolved .= convolve2D.(range,(padded_img,), (filter,))`

This means that each element of the blurred image (f_convolved .=) is computed passing an element of the range that was produced by the padding (CartesianIndices ranging from 1:5 in each axis), and the full padded_img and filter (thus the notation (padded_img,) an (filter,)). The algorithm goes like this:

1. Given an index of the original image (.eg, (3,3), whose value is 1), take its neighborhood, which extends 3 to the left and 3 to the right (thus 0:6, 0:6). In the case of the first index ((1,1) whose value i 0), we go -2:4, -2:4.
2. We then compute the view of this element  and its neighborhood, which corresponds to the frame (fr) that needs to be multiplied by the filter (w).
3. Next we multiply fr and w element by element
4. finally we sum

In [252]:
function convolve2D(I, padf, filter) 
    # I rather than range, we will apply this function index by index
    frame = filter_frame(I, size(filter, 1))
    d  = view(padf, frame...)
    sum(d .* filter)  # returns the view corresponding to index I
end

convolve2D (generic function with 1 method)

In [253]:
fcc1 = similar(fxy);

In [254]:
fcc1 .= convolve2D.(cr, (fxyp,), (w,));

In [255]:
fcc1

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  9.0  8.0  7.0  0.0
 0.0  6.0  5.0  4.0  0.0
 0.0  3.0  2.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [256]:
flip2d(fcc1)

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  1.0  2.0  3.0  0.0
 0.0  4.0  5.0  6.0  0.0
 0.0  7.0  8.0  9.0  0.0
 0.0  0.0  0.0  0.0  0.0